<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# EWHA bagging _ alpha (솔빈이 코드 추가)

In [20]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [21]:
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")



In [22]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

## 1. build DB

In [23]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
import numpy as np


UPSTAGE_API_KEY = api_key

# .npy 파일 로드 (타입==넘파이)
ewhaDB = np.load(data_path+f'embedding/full_ewha500.npy')

ewhaDB_embed = np.load(data_path+f'embedding/full_ewha500_embed.npy')
ewhaDB_embed = ewhaDB_embed.tolist()

ewhaDB_embed

[[0.0098114013671875,
  0.0006871223449707031,
  -0.00771331787109375,
  0.00971221923828125,
  0.018707275390625,
  -0.0097503662109375,
  -0.0168304443359375,
  0.0064849853515625,
  0.02294921875,
  0.011566162109375,
  0.0031528472900390625,
  0.005855560302734375,
  -0.00281524658203125,
  0.0150909423828125,
  0.01238250732421875,
  0.01226043701171875,
  -0.001590728759765625,
  0.003818511962890625,
  -0.01739501953125,
  -0.002407073974609375,
  0.006038665771484375,
  0.002857208251953125,
  -0.01149749755859375,
  -0.007183074951171875,
  -0.0017614364624023438,
  -0.0176239013671875,
  -0.0161895751953125,
  0.019287109375,
  0.0132904052734375,
  -0.0168914794921875,
  -0.00962066650390625,
  -0.00799560546875,
  -0.00962066650390625,
  -0.0239715576171875,
  -0.0224761962890625,
  -0.016265869140625,
  0.01270294189453125,
  0.0143890380859375,
  0.027252197265625,
  0.01047515869140625,
  -0.01149749755859375,
  -0.00548553466796875,
  0.00812530517578125,
  0.0068702697

## 3. test set 갖고오기

In [24]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [25]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))
testdata = pd.read_csv(data_path+'test_own_ewha.csv')

In [26]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3', 'top1_1pred','top1_2pred','top1_3pred', 'top2pred', 'top3pred', 'predict' ])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = query_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a}

    except :
        print(f'pass: {index}')
        continue 


In [27]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3,top1_1pred,top1_2pred,top1_3pred,top2pred,top3pred,predict
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4. Prompt engineering

In [28]:
import numpy as np


for idx, row in nowtest.iterrows() : # 질문 받아오기 

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(ewhaDB_embed).T).argsort()[::-1]

    nowtest.loc[idx, 'top1'] = ewhaDB[sorted_idx[0]]
    nowtest.loc[idx, 'top2'] = ewhaDB[sorted_idx[1]]
    nowtest.loc[idx, 'top3'] = ewhaDB[sorted_idx[2]]


/tmp/ipykernel_3887407/3603593849.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다. (개정 1996.2.15., 2017.2.8.)
② 학점취득특별시험의 방법, 종류에 관한 세부 사항은 총장이 따로 정한다. (개정
2001.9.24)제35조의3(학점의 인정) ① 다음 각 호의 경우에 취득한 학점은 총장의 승인을 얻어 졸업에
필요한 학점의 2분의 1의 범위 안에서 이를 본교에서 취득한 것으로 본다. (개정
2012.12.31.)1. 재학 중 국내외의 다른 학교에서 학점을 취득한 경우
2. 입학 전 국내외의 고등학교와 「고등교육법」 제2조 각 호의 학교에서 대학교육과정에 상
당하는 교과목을 이수한 경우
② 학점의 인정범위 등 제1항의 시행에 필요한 사항은 총장이 따로 정한다. (개정
1998.6.23)③ 삭제 (1998.6.23.)제35조의4(언어교육원 수강과목 학점의 인정) 입학 전 본교 언어교육원에서 소정의 과정을 이
수한 경우 입학 후 대학의 학점으로 인정할 수 있다. 학점인정대상자 및 범위' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, 'top1'] = ewhaDB[sorted_idx[0]]
/tmp/ipykernel_3887407/3603593849.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.6.17.]제9장 교육과정, 이

In [29]:
try : del [[bagging_pred]]
except : pass
bagging_pred = pd.DataFrame(columns=['questionNum', 'answer', 'top1_1pred', 'top1_2pred', 'top1_3pred', 'top2pred', 'top3pred', 'predict'])

# --------------------qa_chain---------------------------

In [ ]:
# # Step 4: Prompt Templates with One-Shot Example

# from langchain.prompts import PromptTemplate

# qa_prompt_template = """
#     You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.

#     Context: {context}

#     Question: {question}

#     Final Answer:
# """

# # Prompt for context retrieval and feedback
# qa_prompt = PromptTemplate(input_variables=["context", "query"], template=qa_prompt_template)



# # FAISS 벡터 스토어 생성
# vector_store = FAISS.from_embeddings(
#     text_embeddings=text_embedding_pairs,  # 텍스트-임베딩 쌍 리스트
#     embedding=embeddings  # Embeddings 객체 전달
# )


# # Step 5: RAG-based QA System
# retriever = vector_store.as_retriever(search_kwargs={"k": 5})
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     chain_type="stuff",  # "stuff" processes all context at once
#     chain_type_kwargs={"prompt": qa_prompt}  # Pass custom prompt
# )

In [31]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatUpstage(
    api_key=api_key,
    temperature=0, 
    max_tokens=2048,
    top_p=0.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
)


prompt_template = PromptTemplate.from_template(
    '''
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.

    Context: {context}

    Question: {question}

    Final Answer:
        
    '''

)

qa_chain = prompt_template | llm



In [32]:
############# PREDICTION ##########

for idx, row in nowtest.iterrows() :
    #if idx == 50 : break
        
    response = qa_chain.invoke({"question": row.prompts, "context": row.top1})

    if isinstance(response, dict): answer_text = response.get('content', response.get('text', str(response)))
    else: answer_text = str(response)

    nowtest.loc[idx, 'top1pred'] = answer_text.strip()
    generated_answer = extract_answer(answer_text.strip())
    bagging_pred.loc[len(bagging_pred)] = {'questionNum': row.question, 'answer': row.answers, 'top1_1pred': generated_answer}

#
    context = row.top1 + ' ' + row.top2
    response = qa_chain.invoke({"question": row.prompts, "context": context})

    if isinstance(response, dict): answer_text = response.get('content', response.get('text', str(response)))
    else: answer_text = str(response)

    nowtest.loc[idx, 'top2pred'] = answer_text.strip()
    generated_answer = extract_answer(answer_text.strip())
    bagging_pred.loc[idx, f'top2pred'] = generated_answer

#
    context = row.top1 + ' ' + row.top2 + ' ' + row.top3
    response = qa_chain.invoke({"question": row.prompts, "context": context})

    if isinstance(response, dict): answer_text = response.get('content', response.get('text', str(response)))
    else: answer_text = str(response)

    nowtest.loc[idx, 'top3pred'] = answer_text.strip()
    generated_answer = extract_answer(answer_text.strip())
    bagging_pred.loc[idx, f'top3pred'] = generated_answer



for i in range(2,4) : 
    for idx, row in nowtest.iterrows() :
        #if idx == 50 : break
            
        response = qa_chain.invoke({"question": row.prompts, "context": row.top1})

        if isinstance(response, dict): answer_text = response.get('content', response.get('text', str(response)))
        else: answer_text = str(response)

        nowtest.loc[idx, f'top1_{i}pred'] = answer_text.strip()
        generated_answer = extract_answer(answer_text.strip())
        bagging_pred.loc[idx, f'top1_{i}pred'] = generated_answer
        



/tmp/ipykernel_3887407/409691007.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'content='(D) 3학점' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 751, 'total_tokens': 759, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-mini-240612', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-441d2295-e5d6-4698-94c3-1b775a1c83c0-0' usage_metadata={'input_tokens': 751, 'output_tokens': 8, 'total_tokens': 759, 'input_token_details': {}, 'output_token_details': {}}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, 'top2pred'] = answer_text.strip()
/tmp/ipykernel_3887407/409691007.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'D' has dt

# --------------------------------------------------

In [33]:
bagging_pred

,questionNum,answer,top1_1pred,top1_2pred,top1_3pred,top2pred,top3pred,predict
0,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),A,A,A,D,D,NaN
1,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,(E),E,E,E,E,E,NaN
2,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,(B),B,B,B,B,B,NaN
3,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,(B),B,B,B,B,B,NaN
4,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),D,D,D,D,D,NaN
5,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,(D),D,D,D,D,D,NaN
6,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,(C),C,C,C,C,C,NaN
7,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),A,A,A,A,A,NaN
8,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),D,D,D,D,D,NaN
9,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,(B),B,B,B,B,B,NaN


# bagging

In [34]:
from collections import Counter

final_pred = []
for idx, row in bagging_pred.iterrows() :

    pred = []
    pred.append(row.top1_1pred)
    pred.append(row.top1_2pred)
    pred.append(row.top1_3pred)
    pred.append(row.top2pred)
    pred.append(row.top3pred)

    counts = Counter(pred)

    prediction = counts.most_common(1)[0][0]
    if prediction == None :
        try : prediction = counts.most_common(2)[1][0]
        except : pass
    final_pred.append(prediction)
    bagging_pred.loc[idx, 'predict'] = prediction

# final_pred

/tmp/ipykernel_3887407/908629431.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bagging_pred.loc[idx, 'predict'] = prediction


In [35]:
######### 정답 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, final_pred)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        bagging_pred.loc[idx, 'iswrong'] = '-'
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        bagging_pred.loc[idx, 'iswrong'] = '-'
        
acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("wrong:", wrong)
bagging_pred.loc[len(bagging_pred), 'predict'] = acc

----------
A
idx: 0 | generated answer: A, answer: (D)
----------
E
idx: 1 | generated answer: E, answer: (E)
----------
B
idx: 2 | generated answer: B, answer: (B)
----------
B
idx: 3 | generated answer: B, answer: (B)
----------
D
idx: 4 | generated answer: D, answer: (C)
----------
D
idx: 5 | generated answer: D, answer:  (D)
----------
C
idx: 6 | generated answer: C, answer:  (C)
----------
A
idx: 7 | generated answer: A, answer:  (D)
----------
D
idx: 8 | generated answer: D, answer:  (C)
----------
B
idx: 9 | generated answer: B, answer:  (B)
----------
D
idx: 10 | generated answer: D, answer:  (B)
----------
C
idx: 11 | generated answer: C, answer:  (C)
----------
B
idx: 12 | generated answer: B, answer:  (B)
----------
C
idx: 13 | generated answer: C, answer:  (E)
----------
A
idx: 14 | generated answer: A, answer:  (A)
----------
C
idx: 15 | generated answer: C, answer:  (C)
----------
E
idx: 16 | generated answer: E, answer:  (E)
----------
B
idx: 17 | generated answer: B, an

In [36]:
bagging_pred

,questionNum,answer,top1_1pred,top1_2pred,top1_3pred,top2pred,top3pred,predict,iswrong
0,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),A,A,A,D,D,A,-
1,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,(E),E,E,E,E,E,E,NaN
2,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,(B),B,B,B,B,B,B,NaN
3,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,(B),B,B,B,B,B,B,NaN
4,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),D,D,D,D,D,D,-
5,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,(D),D,D,D,D,D,D,NaN
6,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,(C),C,C,C,C,C,C,NaN
7,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),A,A,A,A,A,A,-
8,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),D,D,D,D,D,D,-
9,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,(B),B,B,B,B,B,B,NaN


In [ ]:
#bagging_pred.to_csv(results_path+'ewha_bagging_alpha02.csv')